In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
print(os.listdir("./cell_images/"))
from keras.models import Sequential


['Parasitized', 'Uninfected']


Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD

In [9]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Showing a sample image

In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.3,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory('./cell_images/',
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='training')
test_set = train_datagen.flow_from_directory('./cell_images/',
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 19292 images belonging to 2 classes.
Found 8266 images belonging to 2 classes.


In [13]:
classifier.fit_generator(train_set,
                           steps_per_epoch=345,
                           epochs=10,
                           validation_data=test_set,
                           validation_steps=86)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
345/345 [==============================] - 39s 112ms/step - loss: 0.6675 - acc: 0.5878 - val_loss: 0.5841 - val_acc: 0.7558
Epoch 2/10
345/345 [==============================] - 39s 112ms/step - loss: 0.4312 - acc: 0.8304 - val_loss: 0.2836 - val_acc: 0.9004
Epoch 3/10
345/345 [==============================] - 40s 115ms/step - loss: 0.2643 - acc: 0.9013 - val_loss: 0.2225 - val_acc: 0.9052
Epoch 4/10
345/345 [==============================] - 44s 126ms/step - loss: 0.2254 - acc: 0.9183 - val_loss: 0.2090 - val_acc: 0.9310
Epoch 5/10
345/345 [==============================] - 41s 118ms/step - loss: 0.2282 - acc: 0.9282 - val_loss: 0.2007 - val_acc: 0.9379
Epoch 6/10
345/345 [==============================] - 40s 115ms/step - loss: 0.1956 - acc: 0.9400 - val_loss: 0.2053 - val_acc: 0.9331
Epoch 7/10
345/345 [==============================] - 43s 125ms/step - loss: 0.1882 - acc: 0.9409 - val_loss: 0.1972 - val_acc: 0.9397
Epoch 8

saving the model

In [14]:
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Loading

In [15]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [19]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate_generator(generator=test_set,steps=100)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 94.47%
